In [1]:
import warnings
warnings.filterwarnings('ignore')

import get_compounds_each_provider as import_compounds
import pandas as pd
import sys
sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors
import numpy as np


# WikiData

From here: https://github.com/yuvipanda/python-wdqs/blob/master/wdqs/client.py

##Compound properties table
From here: https://www.wikidata.org/wiki/Wikidata:List_of_properties/Terms

24305 compounds

15413 with a unique structural identifier

Has relationships that link to the largest database on earth

Often no structural description exists, but a PubChemCI does.  


In [ ]:
terms_to_keep = ['smiles','inchi','source_database','ROMol','common_name','Definition', 'synonyms','pubchem_compound_id','lipidmaps_id','metacyc_id','hmdb_id','img_abc_id','chebi_id','kegg_id']

In [ ]:
import_compounds = reload(import_compounds)
wikidata = import_compounds.get_wikidata(terms_to_keep)

In [ ]:
df = wikidata[terms_to_keep]


In [ ]:
wikidata.head()
# compoundLabel

# Add the 590 MetAtlas compounds that are missing from all these databases

In [ ]:

# See below, I did it at the end.


# metatlas_added = pd.read_csv('metatlas_compounds_not_found.csv',index_col=0)
# metatlas_added = metatlas_added[['inchi','name']]
# df = pd.concat([df,metatlas_added], axis=0, ignore_index=True)

# metatlas_added.head()

# miBIG (using notebook and pubchem API)

In [ ]:

def strip_whitespace(x):
    return x.strip()

miBIG = pd.read_pickle('miBIG.pkl')
miBIG = miBIG[~miBIG.common_name.str.contains('Status: 404')] #catch any defunct ids and throw them away here
miBIG.inchi = miBIG.inchi.apply(strip_whitespace)
miBIG['source_database'] = 'miBIG'
df = pd.concat([df,miBIG], axis=0, ignore_index=True)
miBIG.head()


# get names for ones that are missing names (determined from first pass through)

In [ ]:
no_name_cid = pd.read_pickle('no_names_with_cid.pkl')
no_name_cid = no_name_cid[~no_name_cid.common_name.str.contains('Status: 404')] #catch any defunct ids and throw them away here
no_name_cid.inchi = no_name_cid.inchi.apply(strip_whitespace)

df = pd.concat([df,no_name_cid], axis=0, ignore_index=True)
no_name_cid.head()


# get names for missing from inchikey (no CID, determined from first pass through)

In [ ]:
no_name_inchikey = pd.read_pickle('no_names_with_inchikey.pkl')
no_name_inchikey = no_name_inchikey[~no_name_inchikey.common_name.str.contains('Status: 404')] #catch any defunct ids and throw them away here
no_name_inchikey.inchi = no_name_inchikey.inchi.apply(strip_whitespace)

df = pd.concat([df,no_name_inchikey], axis=0, ignore_index=True)
no_name_inchikey.head()


# IMG-ABC

https://img.jgi.doe.gov/cgi-bin/abc/main.cgi?section=NaturalProd&page=smListing

Exported as tab delimited table

1109 Compounds

666 Compounds have a unique structure



In [ ]:
img_abc = import_compounds.get_img(terms_to_keep)
df = pd.concat([df,img_abc[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
img_abc.head()
# Secondary Metabolite (SM) Name
# SM ID

# ENZO Library

502 compounds

501 with unique structures

request table from here:
http://www.enzolifesciences.com/BML-2865/screen-well-natural-product-library/
BML-2865.xlsx

I had to save as tab delimted text.  The Excel JChem structures messed up the excel import.

In [ ]:
enzo = import_compounds.get_enzo(terms_to_keep)
df = pd.concat([df,enzo[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
enzo.head()
# Name 

# MSMLS-Library

In [ ]:
import_compounds = reload(import_compounds)
msmls = import_compounds.get_msmls(terms_to_keep)
df = pd.concat([df,msmls[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
msmls.keys()

In [ ]:
msmls.head()# CNAME
# PC_CID

# MetaCyc

12,370 compounds

11,256 compounds with unique structure

Login as bpbowen@lbl.gov

Smart table is called: "MetAtlas Export MetaCyc Compounds"

Has mapping to reactions

Has compound ontology

Save as spreadsheet from their website. Prior to saving, I deleted the first row because I had to open it in excel ad save as "xlsx" for pandas to read it.  The first row has columns that are longer than Excel's maximum number of characters and they get wrapped into new rows.


In [ ]:
import_compounds = reload(import_compounds)
metacyc = import_compounds.get_metacyc(terms_to_keep)
df = pd.concat([df,metacyc[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
metacyc.head()
# KEGG
# PubChem
# Common-Name
# Names
# Object ID


#Dr. Dukes Phytochemical and Ethnobotanical Database

29585 Compounds

No structural information or ids from external databases.

In [ ]:
# dr_dukes = import_compounds.get_dr_dukes()

# LipidMaps

40716 compounds

40145 with a unique structural description

wget http://lipidmaps.org/resources/downloads/LMSDFDownload28Jun15.tar.gz

Its probably possible to get the enzyme-reaction mapping for these!

In [ ]:
print df.shape
print df.source_database.unique().shape
print df.keys()
df.head()

In [ ]:
import_compounds = reload(import_compounds)
lipid_maps = import_compounds.get_lipid_maps(terms_to_keep)
df = pd.concat([df,lipid_maps[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
lipid_maps.head()
# PUBCHEM_CID
# KEGG_ID
# COMMON_NAME
# SYNONYMS
# ID

# HMDB

41895

41722 neutral and unique structures


wget http://www.hmdb.ca/system/downloads/current/structures.zip

unzip to structures.sdf

has calcualted physical properties from jchem

RDKit can't convert the mol for Cyanocobalmin and Hydroxocobalamin.


In [ ]:
hmdb = import_compounds.get_hmdb(terms_to_keep)
df = pd.concat([df,hmdb[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
hmdb.head()
# GENERIC_NAME
# SYNONYMS
# HMDB_ID

# ChEMBL

ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/releases/chembl_21/chembl_21.sdf.gz

In [ ]:
mz = 116.070608
delta_mz = mz*5/1e6
min_mz = mz - delta_mz
max_mz = mz + delta_mz
print mz
print delta_mz
print min_mz
print max_mz

In [ ]:
#This needs a metadata file to accompany it.
#chembl = pd.read_pickle('/project/projectdirs/openmsi/projects/compound_data/chembl/chembl.pkl')
#These are the fields in the pkl
#(1583839, 3)
#Index([u'ID', u'ROMol', u'chembl_id'], dtype='object')
# chembl = import_compounds.get_chembl()
# df = pd.concat([df,chembl[terms_to_keep]], axis=0, ignore_index=True)
# has 1.583 million compounds
# tackle this next time

# ChEBI

wget ftp://ftp.ebi.ac.uk/pub/databases/chebi/SDF/ChEBI_complete_3star.sdf.gz

wget ftp://ftp.ebi.ac.uk/pub/databases/chebi/SDF/ChEBI_complete.sdf.gz


In [ ]:
chebi = import_compounds.get_chebi(terms_to_keep)
df = pd.concat([df,chebi[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
chebi.head()
# ChEBI Name
# ChEBI ID
# KEGG COMPOUND Database Links
# Synonyms

# GNPS

9312 spectra

3971 chemical structures

wget ftp://ccms-ftp.ucsd.edu/Spectral_Libraries/ALL_GNPS.mgf

Many of the named compounds don't have strucutral identifiers or ids that map to other databases.  Many of the structural identifiers are formated wrong or in the wrong field (example: SMILES are used in the InChI field).


In [ ]:
gnps = import_compounds.get_gnps(terms_to_keep)
df = pd.concat([df,gnps[terms_to_keep]], axis=0, ignore_index=True)

In [ ]:
gnps.head()
# name


# BNICE

http://pubs.acs.org/doi/abs/10.1021/acssynbio.6b00054

http://bioinformatics.oxfordjournals.org/content/21/8/1603.short

SDF Files are here:
http://lincolnpark.chem-eng.northwestern.edu/release/

In [ ]:
df.source_database.unique()

# Load the dataframe and make an rdkit mol for each comound

In [ ]:
df.to_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/compounds_fixedStereo_notchembl.pkl')


In [ ]:
# df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/compounds_fixedStereo_notchembl.pkl')

# Replace float(nan) with ''

In [ ]:
df = df.where((pd.notnull(df)), None)


# Missing the "InChI=" at the start

This is how most of the wikidata inchis are stored.  GNPS has some like this too.

In [ ]:
idx = (df['inchi'].str.startswith('1')) & (df['inchi'])
print sum(idx)
print df.source_database[idx].unique()
df['inchi'][idx] = 'InChI=' + df['inchi']

# Missing the "I" in "InChI"

In [ ]:
idx = (df['inchi'].str.startswith('nChI')) & (df['inchi'])
print sum(idx)
print df.source_database[idx].unique()
df['inchi'][idx] = 'I' + df['inchi']

#has an inchikey instead of inchi

In [ ]:
idx = (df['inchi'].str.endswith('-N')) & (df['inchi'])
print sum(idx)
print df.source_database[idx].unique()
df['inchi'][idx] = ''

# Has an "N/A"

In [ ]:
idx = (df['inchi'].str.startswith('N/A')) & (df['inchi'])
print sum(idx)
print df.source_database[idx].unique()
df['inchi'][idx] = ''

In [ ]:
idx = (df['smiles'].str.startswith('N/A')) & (df['smiles'])
print sum(idx)
print df.source_database[idx].unique()
df['smiles'][idx] = ''

# Has something else in InChI field.  Usually this is SMILES string

In [ ]:
idx = (df['inchi'].str.contains('^((?!InChI).)*$')) & df['inchi']
print sum(idx)
print df.source_database[idx].unique()
df['smiles'][idx] = df['inchi'][idx]
df['inchi'][idx] = ''
df[idx].head()

In [ ]:
def make_mol_from_smiles_and_inchi(row):
    if not row['ROMol']:
        mol= []
        if row['inchi']:
            mol = Chem.MolFromInchi(row['inchi'].encode('utf-8'))
        elif row['smiles']:
            mol = Chem.MolFromSmiles(row['smiles'].encode('utf-8'))
        if mol:
            return mol
    else:
        return row['ROMol']

df.ROMol = df.apply(make_mol_from_smiles_and_inchi, axis=1)

# Take a look at the rows that don't have an rdkit mol.
* They don't have a SMILES or InChI
* RDKit could not parse their description

# Desalt and remove disconnected components

In [ ]:
def desalt_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()>1:
            c = import_compounds.desalt(x)
            if c[1]:
                return c[0]
            else:
                return x
    
df.ROMol = df.ROMol.apply(desalt_compounds_in_dataframe)


# neutralize those that are charged

In [ ]:
import_compounds = reload(import_compounds)
def neutralize_compounds_in_dataframe(x):
    if x:
        if x.GetNumAtoms()> 0:
            neutral_mol = []
            try:
                c = import_compounds.NeutraliseCharges(x)
                neutral_mol = c[0]
            except:
                pass
            if neutral_mol:
                return neutral_mol
            
df.ROMol = df.ROMol.apply(neutralize_compounds_in_dataframe)

In [ ]:
def calculate_num_radicals_in_dataframe(x):
    num_radicals = 0.0
    if x:
        num_radicals = Descriptors.NumRadicalElectrons(x)        
    return num_radicals

def calculate_formula_in_dataframe(x):
    formula = ''
    if x:
        formula = rdMolDescriptors.CalcMolFormula(x)
    return formula

def calculate_monoisotopic_mw_in_dataframe(x):
    mw = 0.0
    if x:
        mw = Descriptors.ExactMolWt(x)
    return mw

def calculate_inchi_in_dataframe(x):
    inchi = ''
    if x:
        try:
            inchi = Chem.MolToInchi(x)
        except:
            pass#This fails when can't kekulize mol
    return inchi

def calculate_flattened_inchi_in_dataframe(x):
    flattened_inchi = ''
    if x:
        sm = Chem.MolToSmiles(x).replace('@','')
        flattened_rdkit_mol = Chem.MolFromSmiles(sm)
        try:
            flattened_inchi = Chem.MolToInchi(flattened_rdkit_mol)
        except:
            pass#This fails when can't kekulize mol
    return flattened_inchi

def calculate_inchikey_in_dataframe(x):
    ik = ''
    if x:
        try:
            ik = Chem.InchiToInchiKey(x)
        except:
            pass#This fails when can't kekulize mol.  Carbo-cations are the culprit usually. 
    return ik

def calculate_charge_in_dataframe(x):
    if x:
        my_charge = Chem.GetFormalCharge(x)
        return my_charge
            
df['charge'] = df.ROMol.apply(calculate_charge_in_dataframe)
df['formula'] = df.ROMol.apply(calculate_formula_in_dataframe)
df['monoisotopic_mw'] = df.ROMol.apply(calculate_monoisotopic_mw_in_dataframe)
df['num_radicals'] = df.ROMol.apply(calculate_num_radicals_in_dataframe)

df['metatlas_inchi'] = df.ROMol.apply(calculate_inchi_in_dataframe)
df['metatlas_inchi_key'] = df.metatlas_inchi.apply(calculate_inchikey_in_dataframe)

df['flat_inchi'] = df.ROMol.apply(calculate_flattened_inchi_in_dataframe)
df['flat_inchikey'] = df.flat_inchi.apply(calculate_inchikey_in_dataframe)


# make venn

In [ ]:
# dbs = df.source_database.unique().tolist()
# M = np.zeros((len(dbs),len(dbs)))

# for i in range(len(dbs)):
# #     df1 = df.loc[df['source_database'] == dbs[i]]
# #     M[i,0] = df1.shape[0]
# #     print dbs
#     for j in range(0,len(dbs)):
#         #i is row, j is column
#         u1 = df.loc[df['source_database'] == dbs[i],'metatlas_inchi_key'].unique().tolist()
#         u2 = df.loc[df['source_database'] == dbs[j],'metatlas_inchi_key'].unique().tolist()
#         u1u2 = set(u1).intersection(u2)
#         M[i,j] = len(u1u2)
# M.astype(int)

# create consolidate table from unique compounds

In [ ]:
df.to_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/all_stereo_compounds_temp.pkl')
# df.to_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/all_compounds_temp.pkl')

In [2]:
df = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/all_stereo_compounds_temp.pkl')

In [3]:
df.source_database.unique()

array(['wikidata', None, 'img', 'enzo', 'msmls', 'metacyc', 'lipidmaps',
       'hmdb', 'chebi', 'gnps'], dtype=object)

In [4]:
def strip_non_ascii(string):
    ''' Returns the string without non ASCII characters'''
    stripped = (c for c in string if 0 < ord(c) < 127)
    return ''.join(stripped)

def make_strings_consistent(x):
    if x:
        if type(x) == list:
            x = '///'.join([strip_non_ascii(s) for s in x])
        else:
            x = strip_non_ascii(x) 
    return x

In [5]:
df = df[df.metatlas_inchi_key != '']

In [6]:

    #    compound.name = unicode(df_row.common_name, "utf-8",errors='ignore')

# def make_strings_consistent(x):
#     try:
#         if type(x) == list:
#             x = [str(s.encode('utf-8')) for x in x]
#         else:
#             x = str(x.encode('utf-8'))
#     except:
#         if type(x) == list:
#             x = [str(s) for x in x]
#         else:
#             x = str(x)
#     return x



df['common_name'] = df['common_name'].apply(make_strings_consistent)
df['synonyms'] = df['synonyms'].apply(make_strings_consistent)

In [8]:
my_keys = df.keys().tolist()
my_keys.remove('smiles')
my_keys.remove('inchi')
my_keys.remove('ROMol')

In [9]:
for k in my_keys:
    print k
    df[k] = df[k].astype(str)#apply(str)

Definition
chebi_id
common_name
hmdb_id
img_abc_id
kegg_id
lipidmaps_id
metacyc_id
pubchem_compound_id
source_database
synonyms
charge
formula
monoisotopoic_mw
num_radicals
metatlas_inchi
metatlas_inchi_key
flat_inchi
flat_inchikey


# Remove URLS from various fields throughout

In [10]:
for k in my_keys:
    print k
    df[k] = df[k].str.replace(r'<[^>]*>', '')

Definition
chebi_id
common_name
hmdb_id
img_abc_id
kegg_id
lipidmaps_id
metacyc_id
pubchem_compound_id
source_database
synonyms
charge
formula
monoisotopoic_mw
num_radicals
metatlas_inchi
metatlas_inchi_key
flat_inchi
flat_inchikey


In [11]:
df.synonyms[100000]

'Gln-Thr///Glutamine Threonine dipeptide///Glutamine-Threonine dipeptide///Glutaminylthreonine///L-Glutaminyl-L-Threonine///Q-T Dipeptide///QT dipeptide'

In [12]:
#TODO: define my_keys once! then list the ones to pull out that need special attention
my_agg_keys = my_keys
my_agg_keys.remove('pubchem_compound_id') #has to be handled special...
my_agg_keys.remove('metatlas_inchi_key') #its the grouby key
my_agg_dict = {}

def pubchem_combine_fun(x):
    new_list = []
    for d in x:
        if (d) and (d != 'None'):
            d = str(int(float(d)))
            new_list.append(d)
    return '///'.join(list(set(new_list)))

my_agg_dict['pubchem_compound_id'] = pubchem_combine_fun

for k in my_agg_keys:
    my_agg_dict[k] = lambda x: '///'.join(list(set([d for d in x if (d) and (d != '') and (d.lower() != 'none') and (d.lower() != 'nan')])))

gb = df.groupby('metatlas_inchi_key').agg(my_agg_dict)
gb.reset_index(inplace=True)


# import itertools
# import operator

# def most_common(dL):
#     # get an iterable of (item, iterable) pairs
#     L = dL.split('///')
#     SL = sorted((x, i) for i, x in enumerate(L))
#     # print 'SL:', SL
#     groups = itertools.groupby(SL, key=operator.itemgetter(0))
#     # auxiliary function to get "quality" for an item
#     def _auxfun(g):
#         item, iterable = g
#         count = 0
#         min_index = len(L)
#         for _, where in iterable:
#             count += 1
#             min_index = min(min_index, where)
#             # print 'item %r, count %r, minind %r' % (item, count, min_index)
#             return count, -min_index
#     # pick the highest-count/earliest item
#     return max(groups, key=_auxfun)[0]

# def take_first_common_name(x):
#     return sorted(x.split('///'),key=len)[0]
# gb['common_name'] = gb['common_name'].apply(take_first_common_name)
# gb['common_name'] = gb['common_name'].apply(most_common)

In [13]:
gb.head(10)

,metatlas_inchi_key,pubchem_compound_id,img_abc_id,charge,Definition,metacyc_id,kegg_id,num_radicals,flat_inchi,synonyms,metatlas_inchi,hmdb_id,chebi_id,flat_inchikey,common_name,formula,lipidmaps_id,source_database,monoisotopoic_mw
0,AAABMNXUOFPYQK-WMBKCNCTSA-N,,,0.0,,,C17009,0.0,"InChI=1S/C25H43NO10/c1-13-10-25(32-8,36-15(3)14(13)2)20(28)22(29)26-23-19-18(33-12-34-23)21(31-7)24(4,5)17(35-19)9-16(11-27)30-6/h14-21,23,27-28H,1,9-12H2,2-8H3,(H,26,29)",,"InChI=1S/C25H43NO10/c1-13-10-25(32-8,36-15(3)14(13)2)20(28)22(29)26-23-19-18(33-12-34-23)21(31-7)24(4,5)17(35-19)9-16(11-27)30-6/h14-21,23,27-28H,1,9-12H2,2-8H3,(H,26,29)/t14-,15-,16+,17?,18+,19+,20-,21-,23+,25-/m1/s1",,CHEBI:80858,AAABMNXUOFPYQK-UHFFFAOYSA-N,Mycalamide B,C25H43NO10,,chebi,517.288696576
1,AAABYGXUNQSIIU-IOWSJCHKSA-N,52926334,,0.0,,,,0.0,"InChI=1S/C33H65O10P/c1-3-5-7-9-11-13-15-17-19-21-23-25-33(37)43-31(29-42-44(38,39)41-27-30(35)26-34)28-40-32(36)24-22-20-18-16-14-12-10-8-6-4-2/h30-31,34-35H,3-29H2,1-2H3,(H,38,39)",,"InChI=1S/C33H65O10P/c1-3-5-7-9-11-13-15-17-19-21-23-25-33(37)43-31(29-42-44(38,39)41-27-30(35)26-34)28-40-32(36)24-22-20-18-16-14-12-10-8-6-4-2/h30-31,34-35H,3-29H2,1-2H3,(H,38,39)/t30-,31+/m0/s1",,,AAABYGXUNQSIIU-UHFFFAOYSA-N,PG(13:0/14:0),C33H65O10P,LMGP04010069,lipidmaps,652.43153491
2,AAAFZMYJJHWUPN-SOOFDHNKSA-N,,,0.0,"An organophosphate oxoanion arising from deprotonation of the phosphate OH groups of D-ribose 1,5-diphosphate; major species at pH 7.3.///A ribose bisphosphate that consists of D-ribose having two monophosphate groups at the 1- and 5-positions.",,,0.0,"InChI=1S/C5H12O11P2/c6-3-2(1-14-17(8,9)10)15-5(4(3)7)16-18(11,12)13/h2-7H,1H2,(H2,8,9,10)(H2,11,12,13)","D-Ribofuranose, 1,5-bis(dihydrogen phosphate)///Ribose-1,5-bisphosphate///Ribose-1,5-diphosphate///D-ribose 1,5-bisphosphate///D-ribose 1,5-diphosphate///D-ribose 1,5-diphosphate tetraanion","InChI=1S/C5H12O11P2/c6-3-2(1-14-17(8,9)10)15-5(4(3)7)16-18(11,12)13/h2-7H,1H2,(H2,8,9,10)(H2,11,12,13)/t2-,3-,4-,5?/m1/s1",,CHEBI:17994///CHEBI:58345,AAAFZMYJJHWUPN-UHFFFAOYSA-N,"D-ribose 1,5-bisphosphate///D-ribose 1,5-bisphosphate(4-)",C5H12O11P2,,chebi,309.985484464
3,AAAFZMYJJHWUPN-TXICZTDVSA-N,70678980,,0.0,"A D-ribose 1,5-diphosphate in which the anomeric centre has alpha-configuration.///A D-ribose 1,5-diphosphate(4) in which the anomeric centre has alpha-configuration.",RIBOSE-15-BISPHOSPHATE,C01151,0.0,"InChI=1S/C5H12O11P2/c6-3-2(1-14-17(8,9)10)15-5(4(3)7)16-18(11,12)13/h2-7H,1H2,(H2,8,9,10)(H2,11,12,13)","D-Ribose 1,5-diphosphate///Rib-1,5-P2///Ribose 1,5-bisphosphate///alpha-D-ribose 1,5-diphosphate///&alpha;-D-ribose 1,5-bisphosphate///ribose-1,5-bisphosphate///alpha-D-ribose 1,5-bisphosphate///alpha-D-ribose 1,5-bisphosphate tetraanion///alpha-D-ribose 1,5-diphosphate(4-)","InChI=1S/C5H12O11P2/c6-3-2(1-14-17(8,9)10)15-5(4(3)7)16-18(11,12)13/h2-7H,1H2,(H2,8,9,10)(H2,11,12,13)/t2-,3-,4-,5-/m1/s1",HMDB11688,CHEBI:68688///CHEBI:68819,AAAFZMYJJHWUPN-UHFFFAOYSA-N,"alpha-D-ribose 1,5-bisphosphate///alpha-D-ribose 1,5-bisphosphate(4-)///&alpha;-D-ribose 1,5-bisphosphate///Ribose 1,5-bisphosphate",C5H12O11P2,,chebi///metacyc///hmdb,309.985484464
4,AAAGYPLVPQGIIN-MFEFFIJZSA-N,,,0.0,,,,0.0,"InChI=1S/C27H33FN4O5/c1-16-13-32(17(2)15-33)26(35)21-5-4-6-22(30-25(34)18-7-8-18)24(21)37-23(16)14-31(3)27(36)29-20-11-9-19(28)10-12-20/h4-6,9-12,16-18,23,33H,7-8,13-15H2,1-3H3,(H,29,36)(H,30,34)",,"InChI=1S/C27H33FN4O5/c1-16-13-32(17(2)15-33)26(35)21-5-4-6-22(30-25(34)18-7-8-18)24(21)37-23(16)14-31(3)27(36)29-20-11-9-19(28)10-12-20/h4-6,9-12,16-18,23,33H,7-8,13-15H2,1-3H3,(H,29,36)(H,30,34)/t16-,17+,23-/m0/s1",,CHEBI:124427,AAAGYPLVPQGIIN-UHFFFAOYSA-N,"N-[(2R,3S)-2-[[[(4-fluoroanilino)-oxomethyl]-methylamino]methyl]-5-[(2R)-1-hydroxypropan-2-yl]-3-methyl-6-oxo-3,4-dihydro-2H-1,5-benzoxazocin-10-yl]cyclopropanecarboxamide",C27H33FN4O5,,chebi,512.243498376
5,AAAGYPLVPQGIIN-QZMQVMSPSA-N,,,0.0,,,,0.0,"InChI=1S/C27H33FN4O5/c1-16-13-32(17(2)15-33)26(35)21-5-4-6-22(30-25(34)18-7-8-18)24(21)37-23(16)14-31(3)2

In [ ]:
gb.synonyms.unique()

In [14]:
no_name = gb[gb.common_name == '']


In [ ]:
no_name[no_name.pubchem_compound_id != ''].pubchem_compound_id.to_csv('pubchem_cid_no_name.csv')
no_name[no_name.pubchem_compound_id == ''].metatlas_inchi_key.to_csv('inchi_key_no_name_no_pubchem.csv')


In [ ]:
gb = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')


# Add old MetAtlas compounds that are not in any of these databases

In [15]:
metatlas_not_found = pd.read_pickle('metatlas_compounds_not_found.pkl')
metatlas_not_found.drop('inchi',axis=1,inplace=True)
metatlas_not_found.rename(columns = {'name':'common_name'}, inplace = True)
metatlas_not_found['source_database'] = 'MetAtlas'
for c in set(gb.keys()) - set(metatlas_not_found.keys()):
    metatlas_not_found[c] = ''
metatlas_not_found.head()

,common_name,charge,formula,monoisotopoic_mw,num_radicals,metatlas_inchi,metatlas_inchi_key,flat_inchi,flat_inchikey,source_database,Definition,img_abc_id,chebi_id,pubchem_compound_id,metacyc_id,kegg_id,hmdb_id,synonyms,lipidmaps_id
162,"(4R,5R,6S,7S)-4,5,6,7-tetrahydroxy-2,8-dioxooctanoate",0,C8H12O8,236.053,0,"InChI=1S/C8H12O8/c9-2-5(12)7(14)6(13)3(10)1-4(11)8(15)16/h2-3,5-7,10,12-14H,1H2,(H,15,16)/t3-,5-,6-,7-/m1/s1",CMRFLDMXLVWBFG-SHUUEZRQSA-N,"InChI=1S/C8H12O8/c9-2-5(12)7(14)6(13)3(10)1-4(11)8(15)16/h2-3,5-7,10,12-14H,1H2,(H,15,16)",CMRFLDMXLVWBFG-UHFFFAOYSA-N,MetAtlas,,,,,,,,,
303,"5-(3-carboxy-3-oxopropenyl)-4,6-dihydroxypyridine-2-carboxylate",0,C10H7NO7,253.022,0,"InChI=1S/C10H7NO7/c12-5(9(15)16)2-1-4-6(13)3-7(14)11-8(4)10(17)18/h1-3H,(H,15,16)(H,17,18)(H2,11,13,14)/b2-1+",BKJRVOGPUXJCOY-OWOJBTEDSA-N,"InChI=1S/C10H7NO7/c12-5(9(15)16)2-1-4-6(13)3-7(14)11-8(4)10(17)18/h1-3H,(H,15,16)(H,17,18)(H2,11,13,14)",BKJRVOGPUXJCOY-UHFFFAOYSA-N,MetAtlas,,,,,,,,,
342,"4&alpha;,14&alpha;-dimethyl-porifersta-8,25(27)-dienol",0,C31H52O,440.402,0,"InChI=1S/C31H52O/c1-9-23(20(2)3)11-10-21(4)24-14-18-31(8)27-13-12-25-22(5)28(32)16-17-29(25,6)26(27)15-19-30(24,31)7/h21-25,28,32H,2,9-19H2,1,3-8H3/t21-,22+,23+,24-,25?,28+,29+,30-,31+/m1/s1",UULZCTIPCNJXAT-SPSGOSLXSA-N,"InChI=1S/C31H52O/c1-9-23(20(2)3)11-10-21(4)24-14-18-31(8)27-13-12-25-22(5)28(32)16-17-29(25,6)26(27)15-19-30(24,31)7/h21-25,28,32H,2,9-19H2,1,3-8H3",UULZCTIPCNJXAT-UHFFFAOYSA-N,MetAtlas,,,,,,,,,
737,lys-FMDP,0,C14H24N4O6,344.17,0,"InChI=1S/C14H24N4O6/c1-24-12(20)6-5-11(19)17-8-10(14(22)23)18-13(21)9(16)4-2-3-7-15/h5-6,9-10H,2-4,7-8,15-16H2,1H3,(H,17,19)(H,18,21)(H,22,23)/b6-5+/t9-,10?/m0/s1",SDVKJLYNAOLHBS-JFJMBRGZSA-N,"InChI=1S/C14H24N4O6/c1-24-12(20)6-5-11(19)17-8-10(14(22)23)18-13(21)9(16)4-2-3-7-15/h5-6,9-10H,2-4,7-8,15-16H2,1H3,(H,17,19)(H,18,21)(H,22,23)",SDVKJLYNAOLHBS-UHFFFAOYSA-N,MetAtlas,,,,,,,,,
764,<i>N</i>-acetyl-demethylphophinothricin,0,C6H13N2O4P,208.061,0,"InChI=1S/C6H13N2O4P/c1-4(9)8-5(6(7)10)2-3-13(11)12/h5,13H,2-3H2,1H3,(H2,7,10)(H,8,9)(H,11,12)/t5-/m0/s1",LCUXJCWDRPBSJS-YFKPBYRVSA-N,"InChI=1S/C6H13N2O4P/c1-4(9)8-5(6(7)10)2-3-13(11)12/h5,13H,2-3H2,1H3,(H2,7,10)(H,8,9)(H,11,12)",LCUXJCWDRPBSJS-UHFFFAOYSA-N,MetAtlas,,,,,,,,,


In [16]:
print gb.keys()
print ' '
print metatlas_not_found.keys()
print ' '
print set(gb.keys()) - set(metatlas_not_found.keys())

Index([u'metatlas_inchi_key', u'pubchem_compound_id', u'img_abc_id', u'charge', u'Definition', u'metacyc_id', u'kegg_id', u'num_radicals', u'flat_inchi', u'synonyms', u'metatlas_inchi', u'hmdb_id', u'chebi_id', u'flat_inchikey', u'common_name', u'formula', u'lipidmaps_id', u'source_database', u'monoisotopoic_mw'], dtype='object')
 
Index([u'common_name', u'charge', u'formula', u'monoisotopoic_mw', u'num_radicals', u'metatlas_inchi', u'metatlas_inchi_key', u'flat_inchi', u'flat_inchikey', u'source_database', u'Definition', u'img_abc_id', u'chebi_id', u'pubchem_compound_id', u'metacyc_id', u'kegg_id', u'hmdb_id', u'synonyms', u'lipidmaps_id'], dtype='object')
 
set([])


In [17]:
combo = pd.concat([metatlas_not_found, gb], ignore_index=True)
print combo.shape, gb.shape, metatlas_not_found.shape


(158506, 19) (157919, 19) (587, 19)


In [23]:
combo.shape

(158493, 19)

In [21]:
combo.to_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')
combo.to_csv('/project/projectdirs/metatlas/projects/uniquecompounds.csv')
combo.to_csv('uniquecompounds.csv')

# Mass values are not high enough precision
# Mass might not be includign the charge

In [ ]:
combo = pd.read_pickle('/project/projectdirs/openmsi/projects/ben_run_pactolus/unique_compounds.pkl')

In [18]:
combo.drop('monoisotopoic_mw', axis=1, inplace=True)


In [19]:
def calculate_monoisotopic_mw_from_inchi(row):
    mw = ''
    rdk_mol = Chem.MolFromInchi(row['metatlas_inchi'])
    if rdk_mol:
        mw = str(float(Descriptors.ExactMolWt(rdk_mol)))
    return mw
combo['monoisotopic_mw'] = combo.apply(calculate_monoisotopic_mw_from_inchi, axis=1)


In [ ]:
combo.keys()

In [20]:
combo = combo[(combo.monoisotopic_mw!='') ]

In [ ]:
# my_str = 'radical'
# gb[gb.common_name.str.contains(r'(?:\s|^)%s(?:\s|$)'%my_str,case=False,regex=True)].head(20)

In [ ]:
# from rdkit.Chem import Descriptors
# print Descriptors.NumRadicalElectrons(Chem.MolFromInchi('InChI=1S/C9H11NO3/c10-8(9(12)13)5-6-1-3-7(11)4-2-6/h1-4,8,11H,5,10H2,(H,12,13)/q+1'))
# print Descriptors.NumRadicalElectrons(Chem.MolFromInchi('InChI=1S/C40H46N8S4/c1-25-19-35-33(21-29(25)23-49-39(45(3)4)46(5)6)41-37(51-35)27-11-15-31(16-12-27)43-44-32-17-13-28(14-18-32)38-42-34-22-30(26(2)20-36(34)52-38)24-50-40(47(7)8)48(9)10/h11-22H,23-24H2,1-10H3/q+2/b44-43+'))

In [ ]:
# import sys
# sys.path.append('/global/project/projectdirs/openmsi/jupyterhub_libs/anaconda/lib/python2.7/site-packages')
# from rdkit import Chem
# import numpy as np
# import pandas as pd
# from rdkit.Chem import Draw
# from rdkit.Chem import PandasTools

# %matplotlib inline


# inchis = ['InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5,6)/p+1/t2-/m1/s1',
#     'InChI=1S/C18H18N4O6S4/c1-3-21-13-7-5-11(31(23,24)25)9-15(13)29-17(21)19-20-18-22(4-2)14-8-6-12(32(26,27)28)10-16(14)30-18/h5-10H,3-4H2,1-2H3,(H,23,24,25)(H,26,27,28)/q+1/p-1',
#          'InChI=1S/C3H7NO2.H3N/c1-2(4)3(5)6;/h2H,4H2,1H3,(H,5,6);1H3/p+1/t2-;/m0./s1',
#           'InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5,6)/t2-/m0/s1',
#           'InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5,6)/t2-/m1/s1']
# labels = ['D-Alanine Cation','2,2''-azino-bis-(3-ethylbenzothiazoline-6-sulfonate) radical cation', 'ammonium alanine','L-alanine','D-Alanine']

In [ ]:


# mols = [Chem.MolFromInchi(ii) for ii in inchis]
# de_salt = [m[0] for m in [desalt(m) for m in mols]]
# neutralized = [m[0] for m in [NeutraliseCharges(m) for m in de_salt]]

# # sm1 = Chem.MolToSmiles(m)#,isomericSmiles=True)
# # m2 = Chem.MolFromSmiles(sm1)

# # sm1 = Chem.MolToInchi(m2)
# # m3 = Chem.MolFromInchi(sm1)

In [ ]:
# Chem.Draw.MolsToGridImage(mols+de_salt+neutralized,
#                           legends = labels+['desalted %s'%m for m in labels]+['neutralized %s'%m for m in labels],
#                           molsPerRow = len(inchis))